# Variational Autoencoders

In [46]:
#imports
from keras.layers import Dense, Activation, Add, Multiply, Input, Reshape, Lambda, Concatenate
from keras.models import Model
from keras.datasets import mnist
import tensorflow_probability as tfp
from keras.utils import plot_model
import tensorflow as tf

#get data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 784) / 255.
x_test = x_test.reshape(-1, 784) / 255.

#set parameters
middle_layer = 256
latent_size = 3
epochs = 10
batch_size = 128


In [5]:
x_train.shape

(60000, 784)

In [47]:
#create model
inp = Input(shape=(784,), name="input")
enc = Dense(middle_layer, activation='relu', name="encoding1")(inp)
enc_sigma = Dense(latent_size, name="encoding2_Sigma")(enc)
enc_mu = Dense(latent_size, name="encoding2_Mu")(enc)
#eps = Input(tensor=tfp.Normal().sample(latent_size))
#tmp = Multiply()([enc_sigma, eps])
#latent = Add()([enc_mu, tmp])
latent = Lambda(lambda ms: tfp.distributions.MultivariateNormalDiag(loc=ms[0], scale_diag=ms[1]).sample(), name="LatentEncoding")([enc_mu, enc_sigma])
dec = Dense(middle_layer, activation='relu', name="decoding1")(latent)
dec = Dense(784, activation='relu', name="decoding2")(dec)
out_dist = Reshape([28, 28], name="Reshape")(dec)
#out_dist_joint = Lambda(lambda decoded: tfp.distributions.Independent(tfp.distributions.Bernoulli(logits=out_dist), 2), output_shape=(1,1), name="BernoulliProbability")(dec)
#we cant do the lambda layer above, because it wont return anything; just instantiates distro. 
#BUT, there exists an actual tfp layer!! So apparently, you CAN return a distribution! :)
out_dist_joint = tfp.layers.IndependentBernoulli(tf.shape(out_dist), tfp.distributions.Bernoulli.logits)
concatenated_latent = Concatenate(axis=1,name="SigmaAndMu")([enc_sigma, enc_mu])
vae_model = Model(inputs=inp, outputs=[out_dist_joint(out_dist), concatenated_latent])

vae_model.summary()
plot_model(vae_model, to_file='vae_model.png')

AttributeError: 'tuple' object has no attribute 'layer'

In [30]:
#we need multiple losses:
def reconstruction_loss(y_)
losses = {"BernoulliProbability" : reconstruction_loss,
          "SigmaAndMu" : KL_loss}

In [ ]:
def nll(y_true, y_pred):
    """ Negative log likelihood (Bernoulli). """

    lh = K.tf.distributions.Bernoulli(probs=y_pred)

    return - K.sum(lh.log_prob(y_true), axis=-1)